##Generacja sygnałów **X** i zbioru zmiennych **y**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import tensorflow as tf
import os
from tensorflow.keras import datasets, layers, models, losses, Model
from keras import backend

In [ ]:
if os.path.isdir('SignalSeparation'):
    !rm -rf {'SignalSeparation'}
!git clone https://github.com/MieszkoP/SignalSeparation.git
from SignalSeparation import program as p

Cloning into 'SignalSeparation'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 50 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [ ]:
size = 50000
X_data = np.zeros([size,1000])
Y_data = np.zeros([size,4])
for i in range(size):
  X_data[i,:], Y_data[i,0], Y_data[i,1], Y_data[i,2], Y_data[i,3] = p.GenerateSignal(np.random.uniform(0,100), 0.8, 15)

In [ ]:
Y_new = p.Standardize(Y_data, 400, 500, 400, 600, 0.8, 15)

Zbiór **treningowy** i **walidacyjny**

In [ ]:
X = X_data

size = Y_new.shape[0]

Y_new_test = Y_new[:int(size*0.1),:]
X_t = X[:int(size*0.1),:]

Y_new = Y_new[int(size*0.1):,:]
X = X[int(size*0.1):,:]

## Utworzenie sieci neuronowej

Funkcja wagi

In [ ]:
CostHeightProp = p.CostFunction(p.RatioOfUniforms, 0.8, 15, 1, p.HeightProportionTensorflow(p.DeStandarizeHeight(Y_new[:,0], 0.8, 15),p.DeStandarizeHeight(Y_new[:,1], 0.8, 15)))
CostPlace2 = p.CostFunction(p.SumOfUniforms, 0.1, 0.9, 1, Y_new[:,3])

Zmodyfikowana funkcja kosztu dla $h_{ratio}$ 

In [ ]:
def mean_squared_error_new_height(y_true, y_pred, y_true1, y_true2):
  y_pred = tf.convert_to_tensor(y_pred)
  y_true = tf.cast(y_true, y_pred.dtype)

  #Destandaryzacja do 0.8 - 15:

  y_true1 = p.DeStandarizeHeight(y_true1, 0.8, 15)
  y_true2 = p.DeStandarizeHeight(y_true2, 0.8, 15)
  
  return backend.mean(tf.math.squared_difference(y_pred, y_true)*CostHeightProp(p.HeightProportionTensorflow(y_true1, y_true2)), axis=-1)

In [ ]:
def height_dif(y_true, y_pred):
  return mean_squared_error_new_height(p.HeightProportionTensorflow(y_true[:,1], y_true[:,0]),p.HeightProportionTensorflow(y_pred[:,1],y_pred[:,0]), y_true[:,0], y_true[:,1])*0.00255

Zmodyfikowana funkcja kosztu dla $t_{m2}$

In [ ]:
def mean_squared_error_new_position2(y_true, y_pred):
  y_pred = tf.convert_to_tensor(y_pred)
  y_true = tf.cast(y_true, y_pred.dtype)

  #Destandaryzacja do 0.8 - 15:
  
  return backend.mean(tf.math.squared_difference(y_pred, y_true)*CostPlace2(y_true), axis=-1)

Funkcja kosztu dla $t_{\Delta}$

In [ ]:
def freq_dif(y_true, y_pred):

  true = p.PlacesChange(y_true[:,3], 0.1, 1.7)-y_true[:,2]
  pred = p.PlacesChange(y_pred[:,3], 0.1, 1.7)-y_pred[:,2]
  
  return tf.keras.losses.mean_squared_error(true,pred)*0.224

Funkcje kosztu dla $h_1$, $h_2$, $t_{m1}$ i dla $t_{m2}$

In [ ]:
def h1loss(y_true, y_pred):
  return tf.keras.losses.mean_squared_error(y_true[:,0],y_pred[:,0])

def h2loss(y_true, y_pred):
  return tf.keras.losses.mean_squared_error(y_true[:,1],y_pred[:,1])

def p1loss(y_true, y_pred):
  return tf.keras.losses.mean_squared_error(y_true[:,2],y_pred[:,2])

def p2loss(y_true, y_pred):
  return mean_squared_error_new_position2(y_true[:,3],y_pred[:,3])

Finalna funkcja kosztu

In [ ]:
def custom_loss_function(y_true, y_pred):
  loss = height_dif(y_true, y_pred)+h1loss(y_true, y_pred)+h2loss(y_true, y_pred)+p1loss(y_true, y_pred)+p2loss(y_true, y_pred)+freq_dif(y_true, y_pred)
  #print(y_true[0,:])
  return loss

Model trenowanej sieci

In [ ]:
def TrainNetwork(seed, fork_point, optim, basic_lr, BatchNormalization):
  tf.random.set_seed(seed)
  
  model = p.CreateNetwork(BatchNormalization, fork_point) #Ustal czy w architekturze sieci będzie normalizacja Batch na początku czy nie

  opt = optim(learning_rate=basic_lr)
  model.compile(loss=custom_loss_function,
                optimizer=opt,
                metrics = [h1loss, h2loss, p1loss, p2loss, height_dif, freq_dif],
                run_eagerly=False)

  earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1, mode='min')
  reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=4, verbose=1, min_lr=0.00001, min_delta=1e-4, mode='min')


  mymodel=model.fit(
      X,
      Y_new,
      batch_size=100,
      epochs=300,
      verbose=1,
      callbacks=[reduce_lr_loss, earlyStopping],
      validation_data = (X_t, Y_new_test)
  )

  print('Koniec uczenia')
  return mymodel

##Algorytm genetyczny

Definicja hiperparametrów i populacji algorytmu genetycznego

In [ ]:
x = p.Populaton(5, 5) #Zdefiniowanie populacji i hiperparametrów 
x.define_feature(1, "random state", [1, 2, 3]) 1
x.define_feature(2, "network fork point", [1,3,5,7])  2
x.define_feature(3, "optimizer", [tf.keras.optimizers.Adam,tf.keras.optimizers.SGD, tf.keras.optimizers.Adagrad])  0
x.define_feature(4, "base learning rate", [0.001, 0.01, 0.05, 0.1])  1
x.define_feature(5, "BatchNormalization", [p.NoBatch, p.OnlyBeforeAct, p.OnlyAfterAct, p.AfterInput, p.AfterInputAndAfterAct, p.AfterInputAndBeforeAct])  2

Wczytywanie wcześniej wytrenowanych hiperparametrów

In [ ]:
x.Load('Stored.csv', 'Current.csv')

Trenowanie algorytmu

In [ ]:
beg=0
for itera in range(10):
  if beg==1:
    x.initialize()
    beg=0
  else:
    x.IndBeg()

  x.Check()
  for i in range(5):
    print(x.CheckIsMetricWrote())
    if not x.CheckIsMetricWrote(): #Sprawdzenie czy nie ma już policzonej metryki
      model = TrainNetwork(x.read(1), x.read(2), x.read(3), x.read(4), x.read(5))
      if not np.isnan(model.history['loss'][-1]):
        x.ReadMetric(-model.history['loss'][-1])
      else:
        x.ReadMetric(-100)
    x.nextInd()

  x.Store() #Zapisywanie w historii
  print('Wynik:')
  print(x.initialized)
  x.Sorting() #Sortowanie
  print('Wynik posortowany:')
  print(x.initialized)
  x.Selection() #Selekcja 
  print('Wynik wyselekcjonowany:')
  print(x.initialized)
  x.Crossing() #Krzyżowanie
  print('Wynik krzyżowaniu:')
  print(x.initialized)
  x.Mutation(0.2) #Mutacja ze wsp. mutacji 20%
  print('Wynik po mutacji:')
  print(x.initialized)
  x.Save()

### Wizualizacja hiperparametrow algorytmu

In [ ]:
import numpy as np
import pandas as pd
table = np.loadtxt('/content/DataStored.txt')
import matplotlib.pyplot as plt

In [ ]:
table2 = table[:,-1]

In [ ]:
tabtab = table2.reshape(int(table2.shape[0]/5),5)

In [ ]:
plt.scatter(np.arange(1,tabtab.shape[0]+1),-tabtab[:,0])

plt.scatter(np.arange(1,tabtab.shape[0]+1),-tabtab[:,1])

plt.scatter(np.arange(1,tabtab.shape[0]+1),-tabtab[:,2])

plt.scatter(np.arange(1,tabtab.shape[0]+1),-tabtab[:,3])

plt.scatter(np.arange(1,tabtab.shape[0]+1),-tabtab[:,4])

plt.ylim(0, 0.07)
plt.ylabel('Wynik funkcji kosztu')
plt.xlabel('Nr generacji')

##Wyniki pracy sieci (trenowanie bez SVR)

In [ ]:
model = TrainNetwork(2, 5, tf.keras.optimizers.Adam, 0.01, p.OnlyAfterAct)

In [ ]:
model.model.save_weights('wagi.h5')

In [ ]:
model.model.load_weights('wagi.h5')

In [ ]:
tf.keras.utils.plot_model(model.model, "model.png", show_shapes=True)

In [ ]:
y_pred_train = model.model.predict(X)
y_pred = model.model.predict(X_t)

In [ ]:
Y_destand = p.DeStandarize(Y_new, 400, 500, 400, 600, 0.8, 15)
Y_test_destand = p.DeStandarize(Y_new_test, 400, 500, 400, 600, 0.8, 15)
Y_pred_test_destand = p.DeStandarize(y_pred, 400, 500, 400, 600, 0.8, 15)
Y_pred_destand = p.DeStandarize(y_pred_train, 400, 500, 400, 600, 0.8, 15)

In [ ]:
from scipy.stats import pearsonr

In [ ]:
Y_destand_l, Y_test_destand_l, Y_pred_test_destand_l, Y_pred_destand_l = p.ExcludeByDistance(400, 10, Y_destand, Y_test_destand, Y_pred_test_destand, Y_pred_destand)

In [ ]:
dane_pred0 = np.array(p.HeightProportionTensorflow(Y_pred_destand_l[:,0], Y_pred_destand_l[:,1]))

###Zbiór treningowy

In [ ]:
arr= np.array([])
string = ['wysokość pierwszego sygnału', 'wysokość drugiego sygnału', 'środek pierwszego sygnału','środek drugiego sygnału']
for i in range(4):
  p.PredRealChart(string[i], Y_pred_destand_l[:,i], Y_destand_l[:,i])
  print(p.R2(Y_pred_destand_l[:,i], Y_destand_l[:,i]))
  arr = np.append(arr, p.R2(Y_pred_destand_l[:,i], Y_destand_l[:,i]))
  print(p.NMSE(Y_pred_destand_l[:,i], Y_destand_l[:,i]))
  arr = np.append(arr, p.NMSE(Y_pred_destand_l[:,i], Y_destand_l[:,i]))
  arr = np.append(arr, p.ZYSK(34, 34)) #Nalezy zmodyfikować w taki sposób by kalkulowalo ZYSK
hr=6
string = ['wysokość wyższ. sygnału / wysokość niższ. sygnału', 'środek pierwszego sygnału - środek drugiego sygnału', 'wys. wyższ. sygnału / wys. niższ. sygnału dla $h_{ratio}$>'+str(hr)]

dane0 = np.array(p.HeightProportionTensorflow(Y_destand_l[:,1],Y_destand_l[:,0]))
dane1 = Y_destand_l[:,3]-Y_destand_l[:,2]

dane_pred0 = p.HeightProportionTensorflow(Y_pred_destand_l[:,0], Y_pred_destand_l[:,1])
dane_pred1 = Y_pred_destand_l[:,3]-Y_pred_destand_l[:,2]

p.PredRealChart(string[0], dane_pred0, dane0)
print(p.R2(dane_pred0, dane0))
arr = np.append(arr, p.R2(dane_pred0, dane0))
print(p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.ZYSK(34, 34))

p.PredRealChart(string[1], dane_pred1, dane1)
print(p.R2(dane_pred1, dane1))
arr = np.append(arr, p.R2(dane_pred1, dane1))
print(p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.ZYSK(34, 34))

newdane, newdane_pred = np.array([]), np.array([])
for i in range(len(dane0)):
  if dane0[i]>hr:
    newdane = np.append(newdane, dane0[i])
    newdane_pred = np.append(newdane_pred, dane_pred0[i])

p.PredRealChart(string[2], newdane_pred, newdane)

print(p.R2(newdane_pred, newdane))
arr = np.append(arr, p.R2(newdane_pred, newdane))
print(p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.ZYSK(34, 34))

###Zapisywanie wyników NMSE dla zmiennych

In [ ]:
arr = arr.reshape(7,3)
arr2 = arr.sum(axis=0)
arr3 = np.vstack((arr, arr2))
np.savetxt("wyniki_zkosztem.txt", arr3, fmt='%.4f', delimiter=' & ')

###Zbiór walidacyjny

In [ ]:
arr= np.array([])
string = ['wysokość pierwszego sygnału', 'wysokość drugiego sygnału', 'środek pierwszego sygnału','środek drugiego sygnału']
for i in range(4):
  p.PredRealChart(string[i], Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i])
  print(p.R2(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.R2(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  print(p.NMSE(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.NMSE(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.ZYSK(34, 34))
hr=6
string = ['wysokość wyższ. sygnału / wysokość niższ. sygnału', 'środek pierwszego sygnału - środek drugiego sygnału', 'wys. wyższ. sygnału / wys. niższ. sygnału dla $h_{ratio}$>'+str(hr)]

dane0 = np.array(p.HeightProportionTensorflow(Y_test_destand_l[:,1],Y_test_destand_l[:,0]))
dane1 = Y_test_destand_l[:,3]-Y_test_destand_l[:,2]

dane_pred0 = p.HeightProportionTensorflow(Y_pred_test_destand_l[:,0], Y_pred_test_destand_l[:,1])
dane_pred1 = Y_pred_test_destand_l[:,3]-Y_pred_test_destand_l[:,2]

p.PredRealChart(string[0], dane_pred0, dane0)
print(p.R2(dane_pred0, dane0))
arr = np.append(arr, p.R2(dane_pred0, dane0))
print(p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.ZYSK(34, 34))

p.PredRealChart(string[1], dane_pred1, dane1)
print(p.R2(dane_pred1, dane1))
arr = np.append(arr, p.R2(dane_pred1, dane1))
print(p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.ZYSK(34, 34))

newdane, newdane_pred = np.array([]), np.array([])
for i in range(len(dane0)):
  if dane0[i]>hr:
    newdane = np.append(newdane, dane0[i])
    newdane_pred = np.append(newdane_pred, dane_pred0[i])

p.PredRealChart(string[2], newdane_pred, newdane)

print(p.R2(newdane_pred, newdane))
arr = np.append(arr, p.R2(newdane_pred, newdane))
print(p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.ZYSK(34, 34))

## Optymalizacja parametrow $\gamma$ i $C$.  

In [ ]:
size = 8000
X_data = np.zeros([size,1000])
Y_data = np.zeros([size,4])
for i in range(size):
  X_data[i,:], Y_data[i,0], Y_data[i,1], Y_data[i,2], Y_data[i,3] = p.GenerateSignal(np.random.uniform(0,100), 0.8, 15)

Y_new = p.Standardize(Y_data, 400, 500, 400, 600, 0.8, 15)

X = X_data

size = Y_new.shape[0]

Y_new_test = Y_new[:int(size*0.5),:]
X_t = X[:int(size*0.5),:]

Y_new = Y_new[int(size*0.5):,:]
X = X[int(size*0.5):,:]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

In [ ]:
model = p.CreateNetwork(p.OnlyAfterAct, 5)
model.load_weights('wagi.h5')

In [ ]:
hvector = Model(inputs=model.input,
                                 outputs=model.layers[-5].output)
pvector = Model(inputs=model.input,
                                 outputs=model.layers[-4].output)

Xp = pvector.predict(X)
Xh = hvector.predict(X)

Xp_t = pvector.predict(X_t)
Xh_t = hvector.predict(X_t)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
# y - cztery kolumny

svr = SVR(epsilon=0.001,  kernel='rbf', C=10, gamma=1)
mor2 = MultiOutputRegressor(svr)
mor2.fit(Xh, Y_new[:, :2])
pred_train1 = mor2.predict(Xh)
pred_test1 = mor2.predict(Xh_t)

mor2.fit(Xp, Y_new[:, 2:])
pred_train2 = mor2.predict(Xp)
pred_test2 = mor2.predict(Xp_t)

In [ ]:
y_pred_train = np.hstack((pred_train1, pred_train2))
y_pred = np.hstack((pred_test1, pred_test2))
Y_destand = p.DeStandarize(Y_new, 400, 500, 400, 600, 0.8, 15)
Y_test_destand = p.DeStandarize(Y_new_test, 400, 500, 400, 600, 0.8, 15)
Y_pred_test_destand = p.DeStandarize(y_pred, 400, 500, 400, 600, 0.8, 15)
Y_pred_destand = p.DeStandarize(y_pred_train, 400, 500, 400, 600, 0.8, 15)

In [ ]:
arr0 = np.zeros((0,9))
arr2 = np.zeros((0,9))
for c in [1, 10, 100, 1000, 10000]:
  for gamma in [0.001, 0.01, 0.1, 1, 5]:
    arr0 = arr2
    print(c, gamma)

    

    svr = SVR(epsilon=0.001,  kernel='rbf', C=c, gamma=gamma)
    mor1 = MultiOutputRegressor(svr)
    mor1.fit(Xh, Y_new[:, :2])
    pred_train1 = mor1.predict(Xh)
    pred_test1 = mor1.predict(Xh_t)


    svr = SVR(epsilon=0.001,  kernel='rbf', C=c, gamma=gamma)
    mor2 = MultiOutputRegressor(svr)
    mor2.fit(Xp, Y_new[:, 2:])
    pred_train2 = mor2.predict(Xp)
    pred_test2 = mor2.predict(Xp_t)

    y_pred_train = np.hstack((pred_train1, pred_train2))
    y_pred = np.hstack((pred_test1, pred_test2))

    Y_destand = p.DeStandarize(Y_new, 400, 500, 400, 600, 0.8, 15)
    Y_test_destand = p.DeStandarize(Y_new_test, 400, 500, 400, 600, 0.8, 15)
    Y_pred_test_destand = p.DeStandarize(y_pred, 400, 500, 400, 600, 0.8, 15)
    Y_pred_destand = p.DeStandarize(y_pred_train, 400, 500, 400, 600, 0.8, 15)

    Y_destand_l, Y_test_destand_l, Y_pred_test_destand_l, Y_pred_destand_l = p.ExcludeByDistance(400, 10, Y_destand, Y_test_destand, Y_pred_test_destand, Y_pred_destand)
    dane_pred0 = np.array(p.HeightProportionTensorflow(Y_pred_destand_l[:,0], Y_pred_destand_l[:,1]))

    arr = np.array([c, gamma])

    
    for i in range(4):
      print(p.R2(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
      print(p.NMSE(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))

      arr = np.hstack([arr, p.NMSE(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i])])

    hr=6

    dane0 = np.array(p.HeightProportionTensorflow(Y_test_destand_l[:,1],Y_test_destand_l[:,0]))
    dane1 = Y_test_destand_l[:,3]-Y_test_destand_l[:,2]

    dane_pred0 = p.HeightProportionTensorflow(Y_pred_test_destand_l[:,0], Y_pred_test_destand_l[:,1])
    dane_pred1 = Y_pred_test_destand_l[:,3]-Y_pred_test_destand_l[:,2]

    print(p.R2(dane_pred0, dane0))
    print(p.NMSE(dane_pred0, dane0))
    arr = np.hstack([arr, p.NMSE(dane_pred0, dane0)])

    print(p.R2(dane_pred1, dane1))
    print(p.NMSE(dane_pred1, dane1))
    arr = np.hstack([arr, p.NMSE(dane_pred1, dane1)])

    #arr = np.array([])

    newdane, newdane_pred = np.array([]), np.array([])
    for i in range(len(dane0)):
      if dane0[i]>hr:
        newdane = np.append(newdane, dane0[i])
        newdane_pred = np.append(newdane_pred, dane_pred0[i])

    print(p.R2(newdane_pred, newdane))
    print(p.NMSE(newdane_pred, newdane))
    arr = np.hstack([arr, p.NMSE(newdane_pred, newdane)])

    #print(arr)
    arr2 = np.vstack([arr0, arr])

In [8]:
df = pd.DataFrame(arr2, columns=['c', 'gamma', 'h1', 'h2', 't_m1', 't_m2', 'h_ratio', 't_delta','h_ratio if h_ratio>6'])

In [ ]:
h1 = df.pivot('c', 'gamma', 'h1')
h2 = df.pivot('c', 'gamma', 'h2')

p1 = df.pivot('c', 'gamma', 'p1')
p2 = df.pivot('c', 'gamma', 'p2')

h_ratio = df.pivot('c', 'gamma', 'h_ratio')
pdelta = df.pivot('c', 'gamma', 'pdelta')
h_ratio6 = df.pivot('c', 'gamma', 'h_ratio6')

In [ ]:
import numpy as np
import seaborn as sns; sns.set_theme(style='white')
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import MaxNLocator
plt.figure(figsize=(25, 10))
plt.subplot(2,4,1)
plt.title('NMSE dla $h_1$')
sns.heatmap(h1, square=True, norm=LogNorm(), cmap='gray')

plt.subplot(2,4,2)
plt.title('NMSE dla $h_2$')
sns.heatmap(h2, square=True, norm=LogNorm(), cmap='gray')

plt.subplot(2,4,3)
plt.title('NMSE dla $t_{m1}$')
sns.heatmap(p1, square=True, norm=LogNorm(), cmap='gray')

plt.subplot(2,4,4)
plt.title('NMSE dla $t_{m2}$')
sns.heatmap(p2, square=True, norm=LogNorm(), cmap='gray')

plt.subplot(2,4,5)
plt.title('NMSE dla $h_{ratio}$')
sns.heatmap(h_ratio, square=True, norm=LogNorm(), cmap='gray')

plt.subplot(2,4,6)
plt.title('NMSE dla $t_{\Delta}$')
sns.heatmap(pdelta, square=True, norm=LogNorm(), cmap='gray')

plt.subplot(2,4,7)
plt.title('NMSE dla $h_{ratio}>6$')
sns.heatmap(h_ratio6, square=True, norm=LogNorm(), cmap='gray')

plt.subplot(2,4,7)
plt.title('Suma NMSE dla wszystkich zmiennych$')
sns.heatmap(h_ratio6, square=True, norm=LogNorm(), cmap='gray')


##Finalna Ocena

In [ ]:
size = 50000
X_data = np.zeros([size,1000])
Y_data = np.zeros([size,4])
for i in range(size):
  X_data[i,:], Y_data[i,0], Y_data[i,1], Y_data[i,2], Y_data[i,3] = p.GenerateSignal(np.random.uniform(0,100), 0.8, 15)

Y_new = p.Standardize(Y_data, 400, 500, 400, 600, 0.8, 15)

X = X_data

size = Y_new.shape[0]

Y_new_test = Y_new[:int(size*0.1),:]
X_t = X[:int(size*0.1),:]

Y_new = Y_new[int(size*0.1):,:]
X = X[int(size*0.1):,:]

In [ ]:
model = p.CreateNetwork(p.OnlyAfterAct, 5)
model.load_weights('wagi.h5')

hvector = Model(inputs=model.input,
                                 outputs=model.layers[-5].output)
pvector = Model(inputs=model.input,
                                 outputs=model.layers[-4].output)

Xp = pvector.predict(X)
Xh = hvector.predict(X)

Xp_t = pvector.predict(X_t)
Xh_t = hvector.predict(X_t)

from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

svr = SVR(epsilon=0.001,  kernel='rbf', C=1, gamma=0.1)
mor2 = MultiOutputRegressor(svr)
mor2.fit(Xh, Y_new[:, :2])
pred_train1 = mor2.predict(Xh)
pred_test1 = mor2.predict(Xh_t)

mor2.fit(Xp, Y_new[:, 2:])
pred_train2 = mor2.predict(Xp)
pred_test2 = mor2.predict(Xp_t)

In [ ]:
y_pred_train = np.hstack((pred_train1, pred_train2))
y_pred = np.hstack((pred_test1, pred_test2))
Y_destand = p.DeStandarize(Y_new, 400, 500, 400, 600, 0.8, 15)
Y_test_destand = p.DeStandarize(Y_new_test, 400, 500, 400, 600, 0.8, 15)
Y_pred_test_destand = p.DeStandarize(y_pred, 400, 500, 400, 600, 0.8, 15)
Y_pred_destand = p.DeStandarize(y_pred_train, 400, 500, 400, 600, 0.8, 15)
Y_destand_l, Y_test_destand_l, Y_pred_test_destand_l, Y_pred_destand_l = p.ExcludeByDistance(400, 10, Y_destand, Y_test_destand, Y_pred_test_destand, Y_pred_destand)
dane_pred0 = np.array(p.HeightProportionTensorflow(Y_pred_destand_l[:,0], Y_pred_destand_l[:,1]))

###Zbiór treningowy

In [ ]:
arr= np.array([])
string = ['wysokość pierwszego sygnału', 'wysokość drugiego sygnału', 'środek pierwszego sygnału','środek drugiego sygnału']
for i in range(4):
  p.PredRealChart(string[i], Y_pred_destand_l[:,i], Y_destand_l[:,i])
  print(p.R2(Y_pred_destand_l[:,i], Y_destand_l[:,i]))
  arr = np.append(arr, p.R2(Y_pred_destand_l[:,i], Y_destand_l[:,i]))
  print(p.NMSE(Y_pred_destand_l[:,i], Y_destand_l[:,i]))
  arr = np.append(arr, p.NMSE(Y_pred_destand_l[:,i], Y_destand_l[:,i]))
  arr = np.append(arr, p.ZYSK(34, 34))
hr=6
string = ['wysokość wyższ. sygnału / wysokość niższ. sygnału', 'środek pierwszego sygnału - środek drugiego sygnału', 'wys. wyższ. sygnału / wys. niższ. sygnału dla $h_{ratio}$>'+str(hr)]

dane0 = np.array(p.HeightProportionTensorflow(Y_destand_l[:,1],Y_destand_l[:,0]))
dane1 = Y_destand_l[:,3]-Y_destand_l[:,2]

dane_pred0 = p.HeightProportionTensorflow(Y_pred_destand_l[:,0], Y_pred_destand_l[:,1])
dane_pred1 = Y_pred_destand_l[:,3]-Y_pred_destand_l[:,2]

p.PredRealChart(string[0], dane_pred0, dane0)
print(p.R2(dane_pred0, dane0))
arr = np.append(arr, p.R2(dane_pred0, dane0))
print(p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.ZYSK(34, 34))

p.PredRealChart(string[1], dane_pred1, dane1)
print(p.R2(dane_pred1, dane1))
arr = np.append(arr, p.R2(dane_pred1, dane1))
print(p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.ZYSK(34, 34))

newdane, newdane_pred = np.array([]), np.array([])
for i in range(len(dane0)):
  if dane0[i]>hr:
    newdane = np.append(newdane, dane0[i])
    newdane_pred = np.append(newdane_pred, dane_pred0[i])

p.PredRealChart(string[2], newdane_pred, newdane)

print(p.R2(newdane_pred, newdane))
arr = np.append(arr, p.R2(newdane_pred, newdane))
print(p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.ZYSK(34, 34))

###Zbiór testowy

In [ ]:
arr= np.array([])
string = ['wysokość pierwszego sygnału', 'wysokość drugiego sygnału', 'środek pierwszego sygnału','środek drugiego sygnału']
for i in range(4):
  p.PredRealChart(string[i], Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i])
  print(p.R2(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.R2(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  print(p.NMSE(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.NMSE(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.ZYSK(34, 34))
hr=6
string = ['wysokość wyższ. sygnału / wysokość niższ. sygnału', 'środek pierwszego sygnału - środek drugiego sygnału', 'wys. wyższ. sygnału / wys. niższ. sygnału dla $h_{ratio}$>'+str(hr)]

dane0 = np.array(p.HeightProportionTensorflow(Y_test_destand_l[:,1],Y_test_destand_l[:,0]))
dane1 = Y_test_destand_l[:,3]-Y_test_destand_l[:,2]

dane_pred0 = p.HeightProportionTensorflow(Y_pred_test_destand_l[:,0], Y_pred_test_destand_l[:,1])
dane_pred1 = Y_pred_test_destand_l[:,3]-Y_pred_test_destand_l[:,2]

p.PredRealChart(string[0], dane_pred0, dane0)
print(p.R2(dane_pred0, dane0))
arr = np.append(arr, p.R2(dane_pred0, dane0))
print(p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.ZYSK(34, 34))

p.PredRealChart(string[1], dane_pred1, dane1)
print(p.R2(dane_pred1, dane1))
arr = np.append(arr, p.R2(dane_pred1, dane1))
print(p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.ZYSK(34, 34))

newdane, newdane_pred = np.array([]), np.array([])
for i in range(len(dane0)):
  if dane0[i]>hr:
    newdane = np.append(newdane, dane0[i])
    newdane_pred = np.append(newdane_pred, dane_pred0[i])

p.PredRealChart(string[2], newdane_pred, newdane)

print(p.R2(newdane_pred, newdane))
arr = np.append(arr, p.R2(newdane_pred, newdane))
print(p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.ZYSK(34, 34))

In [ ]:
arr = arr.reshape(7,3)
arr20 = arr.sum(axis=0)
arr9 = np.vstack((arr, arr20))
np.savetxt("wyniki5.txt", arr9, fmt='%.4f', delimiter=' & ')

##Trenowanie bez modyfikacji funkcji kosztu (dla porównania)

Funkcje kosztu dla $h_1$, $h_2$, $t_{m1}$ i dla $t_{m2}$

In [ ]:
def h1loss(y_true, y_pred):
  return tf.keras.losses.mean_squared_error(y_true[:,0],y_pred[:,0])

def h2loss(y_true, y_pred):
  return tf.keras.losses.mean_squared_error(y_true[:,1],y_pred[:,1])

def p1loss(y_true, y_pred):
  return tf.keras.losses.mean_squared_error(y_true[:,2],y_pred[:,2])

def p2loss(y_true, y_pred):
  return tf.keras.losses.mean_squared_error(y_true[:,3],y_pred[:,3])

def freq_dif(y_true, y_pred):

  true = p.PlacesChange(y_true[:,3], 0.1, 1.7)-y_true[:,2]
  pred = p.PlacesChange(y_pred[:,3], 0.1, 1.7)-y_pred[:,2]
  return tf.keras.losses.mean_squared_error(true,pred)*0.224

def height_dif(y_true, y_pred):
  return tf.keras.losses.mean_squared_error(p.HeightProportionTensorflow(y_true[:,1], y_true[:,0]),p.HeightProportionTensorflow(y_pred[:,1],y_pred[:,0]))*0.00255

In [ ]:
def custom_loss_function(y_true, y_pred):
  loss = height_dif(y_true, y_pred)+h1loss(y_true, y_pred)+h2loss(y_true, y_pred)+p1loss(y_true, y_pred)+p2loss(y_true, y_pred)+freq_dif(y_true, y_pred)
  #print(y_true[0,:])
  return loss

def TrainNetwork(seed, fork_point, optim, basic_lr, BatchNormalization):
  tf.random.set_seed(seed)
  
  model = p.CreateNetwork(BatchNormalization, fork_point) #Ustal czy w architekturze sieci będzie normalizacja Batch na początku czy nie

  opt = optim(learning_rate=basic_lr)
  model.compile(loss=custom_loss_function,
                optimizer=opt,
                metrics = [h1loss, h2loss, p1loss, p2loss, height_dif, freq_dif],
                run_eagerly=False)

  earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1, mode='min')
  reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=4, verbose=1, min_lr=0.00001, min_delta=1e-4, mode='min')


  mymodel=model.fit(
      X,
      Y_new,
      batch_size=100,
      epochs=100,
      verbose=1,
      callbacks=[reduce_lr_loss, earlyStopping],
      validation_data = (X_t, Y_new_test)
  )

  print('Koniec uczenia')
  return mymodel

In [ ]:
model = TrainNetwork(2, 5, tf.keras.optimizers.Adam, 0.01, p.OnlyAfterAct)

In [ ]:
y_pred_train = model.model.predict(X)
y_pred = model.model.predict(X_t)

In [ ]:
Y_destand = p.DeStandarize(Y_new, 400, 500, 400, 600, 0.8, 15)
Y_test_destand = p.DeStandarize(Y_new_test, 400, 500, 400, 600, 0.8, 15)
Y_pred_test_destand = p.DeStandarize(y_pred, 400, 500, 400, 600, 0.8, 15)
Y_pred_destand = p.DeStandarize(y_pred_train, 400, 500, 400, 600, 0.8, 15)
Y_destand_l, Y_test_destand_l, Y_pred_test_destand_l, Y_pred_destand_l = p.ExcludeByDistance(400, 10, Y_destand, Y_test_destand, Y_pred_test_destand, Y_pred_destand)
dane_pred0 = np.array(p.HeightProportionTensorflow(Y_pred_destand_l[:,0], Y_pred_destand_l[:,1]))

In [ ]:
arr= np.array([])
string = ['wysokość pierwszego sygnału', 'wysokość drugiego sygnału', 'środek pierwszego sygnału','środek drugiego sygnału']
for i in range(4):
  p.PredRealChart(string[i], Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i])
  print(p.R2(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.R2(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  print(p.NMSE(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.NMSE(Y_pred_test_destand_l[:,i], Y_test_destand_l[:,i]))
  arr = np.append(arr, p.ZYSK(34, 34))
hr=6
string = ['wysokość wyższ. sygnału / wysokość niższ. sygnału', 'środek pierwszego sygnału - środek drugiego sygnału', 'wys. wyższ. sygnału / wys. niższ. sygnału dla $h_{ratio}$>'+str(hr)]

dane0 = np.array(p.HeightProportionTensorflow(Y_test_destand_l[:,1],Y_test_destand_l[:,0]))
dane1 = Y_test_destand_l[:,3]-Y_test_destand_l[:,2]

dane_pred0 = p.HeightProportionTensorflow(Y_pred_test_destand_l[:,0], Y_pred_test_destand_l[:,1])
dane_pred1 = Y_pred_test_destand_l[:,3]-Y_pred_test_destand_l[:,2]

p.PredRealChart(string[0], dane_pred0, dane0)
print(p.R2(dane_pred0, dane0))
arr = np.append(arr, p.R2(dane_pred0, dane0))
print(p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.NMSE(dane_pred0, dane0))
arr = np.append(arr, p.ZYSK(34, 34))

p.PredRealChart(string[1], dane_pred1, dane1)
print(p.R2(dane_pred1, dane1))
arr = np.append(arr, p.R2(dane_pred1, dane1))
print(p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.NMSE(dane_pred1, dane1))
arr = np.append(arr, p.ZYSK(34, 34))

newdane, newdane_pred = np.array([]), np.array([])
for i in range(len(dane0)):
  if dane0[i]>hr:
    newdane = np.append(newdane, dane0[i])
    newdane_pred = np.append(newdane_pred, dane_pred0[i])

p.PredRealChart(string[2], newdane_pred, newdane)

print(p.R2(newdane_pred, newdane))
arr = np.append(arr, p.R2(newdane_pred, newdane))
print(p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.NMSE(newdane_pred, newdane))
arr = np.append(arr, p.ZYSK(34, 34))

In [ ]:
arr = arr.reshape(7,3)
arr20 = arr.sum(axis=0)
arr9 = np.vstack((arr, arr20))
np.savetxt("wyniki5.txt", arr9, fmt='%.4f', delimiter=' & ')

##Walidacja krzyżowa 10-krotna

In [ ]:
def TrainNetwork(seed, fork_point, optim, basic_lr, BatchNormalization, X, Y_new):
  tf.random.set_seed(seed)
  
  model = p.CreateNetwork(BatchNormalization, fork_point) #Ustal czy w architekturze sieci będzie normalizacja Batch na początku czy nie

  opt = optim(learning_rate=basic_lr)
  model.compile(loss=custom_loss_function,
                optimizer=opt,
                metrics = [h1loss, h2loss, p1loss, p2loss, height_dif, freq_dif],
                run_eagerly=False)

  earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1, mode='min')
  reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=4, verbose=1, min_lr=0.00001, min_delta=1e-4, mode='min')


  mymodel=model.fit(
      X,
      Y_new,
      batch_size=100,
      epochs=300,
      verbose=1,
      callbacks=[reduce_lr_loss, earlyStopping],
      validation_data = (X_t, Y_new_test)
  )

  print('Koniec uczenia')
  return mymodel

In [ ]:
size = 50000
X_data = np.zeros([size,1000])
Y_data = np.zeros([size,4])
for i in range(size):
  X_data[i,:], Y_data[i,0], Y_data[i,1], Y_data[i,2], Y_data[i,3] = p.GenerateSignal(np.random.uniform(0,100), 0.8, 15)

Y_new_all = p.Standardize(Y_data, 400, 500, 400, 600, 0.8, 15)

size = Y_new_all.shape[0]

In [ ]:
for it in np.arange(1,11):
  Y_new_test = Y_new_all[int(size*0.1*(it-1)):int(size*0.1*it),:]
  X_t = X_data[int(size*0.1*(it-1)):int(size*0.1*it),:]

  Y_new = np.vstack((Y_new_all[:int(size*0.1*(it-1)),:], Y_new_all[int(size*0.1*it):,:]))
  X = np.vstack((X_data[:int(size*0.1*(it-1)),:], X_data[int(size*0.1*it):,:]))

  print('\n treningowy: ')
  print(Y_new[-1,:])
  print(Y_new.shape)

  print('\n testowy: ')
  print(Y_new_test.shape)
  print(Y_new_test[-1,:])

  model = TrainNetwork(2, 5, tf.keras.optimizers.Adam, 0.01, p.OnlyAfterAct, X, Y_new)


##Symulacja identyfikacji próbki

In [ ]:
n=20

def GenerateSignal(height_ratio, df, bottom_border, top_border): #Wygeneruj sygnał o ustalonym stosunku wysokości pików (pik2/pik1) i o ustalonej odległości df. Rozmiar pików losowy (ale w taki spoób aby stosunek wysokości sie zgadzał), miejsce tez losowe (od 400 do 500)
  
  if height_ratio<0:
    height1 = np.random.uniform(0.8,15/(abs(height_ratio)+1))
    height2 = height1*(abs(height_ratio)+1)
  else:
    height1 = np.random.uniform(0.8*(abs(height_ratio)+1),15)
    height2 = height1/(abs(height_ratio)+1)


  x = np.arange(0,1000,1)
  place1 = np.random.uniform(400,500)
  place2 = place1+df
  y_f1 = scipy.stats.norm(place1, 100/2.355)
  y1 = y_f1.pdf(x)*100/2.355*(2*3.14)**(1/2)*height1
  y_f2 = scipy.stats.norm(place2, 100/2.355)
  y2 = y_f2.pdf(x)*100/2.355*(2*3.14)**(1/2)*height2
  y = y1+y2
  y = np.random.normal(loc=0, scale=0.005, size=1000)+y
  return y, height1, height2, place1, place2

In [ ]:
def predicting(model_n, mor2a, mor2b, signal):
  inp = np.zeros((1,1000))
  inp[0,:] = signal
  
  

  Xp = pvector.predict(inp)
  Xh = hvector.predict(inp)

  out1 = mor2a.predict(Xh)
  out2 = mor2b.predict(Xp)
  
  out = np.hstack((out1, out2))
  out = p.DeStandarize(out, 400, 500, 400, 600, 0.8, 15)
  return out

In [ ]:
import warnings

warnings.filterwarnings("ignore",category=UserWarning)

for j in range(0,10):
  print("Stosunek wysokości = "+str(j+1))
  plt.figure(figsize=[20,10])
  for i in range(0,10):
    #print(np.log10(j))
    data = GenerateSignal(j,40, 0.8, 15)
    predicted = predicting(model, mor2a, mor2b, data[0])[0,:]

    plt.subplot(2,4,1)
    plt.scatter(data[3], predicted[2])
    plt.xlabel('Prawdziwe położenie 1 piku')
    plt.ylabel('Przewidywane położenie 1 piku')
    plt.plot([400, 500], [400, 500], '--')

    plt.subplot(2,4,2)
    plt.scatter(data[4], predicted[3])
    plt.xlabel('Prawdziwe położenie 2 piku')
    plt.ylabel('Przewidywane położenie 2 piku')
    plt.plot([400, 600], [400, 600], '--')

    plt.subplot(2,4,3)
    plt.scatter(data[1], predicted[0])
    plt.xlabel('Prawdziwa wysokość 1 piku')
    plt.ylabel('Przewidywana wysokość 1 piku')
    plt.plot([0.8, 15], [0.8, 15], '--')

    plt.subplot(2,4,4)
    plt.scatter(data[2], predicted[1])
    plt.xlabel('Prawdziwa wysokość 2 piku')
    plt.ylabel('Przewidywana wysokość 2 piku')
    plt.plot([0.8, 15], [0.8, 15], '--')

    plt.subplot(2,4,7)
    plt.scatter(data[1]/data[2], predicted[0]/predicted[1])
    plt.xlabel('Prawdziwy stosunek wysokości pików')
    plt.ylabel('Przewidywany stosunek wysokości pików')
    plt.ylim(0,10)
    plt.xlim(0,10)
    plt.plot([0, 10], [0, 10], '--')

    plt.subplot(2,4,8)
    plt.scatter(data[4]-data[3], predicted[3]-predicted[2])
    plt.xlabel('Prawdziwa różnica pików')
    plt.ylabel('Przewidywana różnica pików')
    plt.xlim(30,50)
    plt.ylim(0,100)
    #plt.plot([0.8, 15], [0.8, 15], '--')

    #plt.
    plt.subplot(2,2,3)
    plt.plot(data[0])
    plt.xlabel('X')
    plt.ylabel('Sygnał')

  plt.show()

In [ ]:
simulations_height = np.ones((20, 8, 10))
simulations_dif = np.ones((20, 8, 10))

In [ ]:
import warnings

warnings.filterwarnings("ignore",category=UserWarning)

for j in range(0,20):
  print("Stosunek wysokości = "+str(j+1))
  plt.figure(figsize=[20,10])
  for i in range(0,10):
    #print(np.log10(j))
    data = GenerateSignal(j,40, 0.8, 15)
    
    predicted = predicting(model, mor2a, mor2b, data[0])[0,:]
    predicted[3]-predicted[2]
    predicted[0]/predicted[1]

In [ ]:
for height in range(0,10):
  for t_delta in range(0,8):
    for n in range(0,20):
      data = GenerateSignal(height,10*(t_delta+1), 0.8, 15)
      predicted = predicting(model, mor2a, mor2b, data[0])[0,:]
      simulations_height[n, t_delta, height] = predicted[0]/predicted[1]
      simulations_dif[n, t_delta, height] = predicted[3]-predicted[2]